In [192]:
import pandas as pd
import numpy as np

base_path = 'data(li)delete10/'
new_path = 'data(li)delete10/'
# 将 filelist 替换为 exp.csv
filename = 'xun.csv'

# 设置随机种子以确保结果可重现
np.random.seed(435)

# 读取原始数据
print(filename)
data_RNA = pd.read_csv(base_path + filename, index_col=0)  # 使用 read_csv 读取 CSV 文件

# 随机删除10%的样本
print(f"原始样本数量: {len(data_RNA)}")
sample_indices = data_RNA.index.tolist()
delete_count = int(len(sample_indices) * 0.1)  # 计算要删除的样本数量

# 随机选择要删除的样本索引
indices_to_delete = np.random.choice(sample_indices, size=delete_count, replace=False)
data_RNA_filtered = data_RNA.drop(indices_to_delete)

print(f"删除样本数量: {delete_count}")
print(f"剩余样本数量: {len(data_RNA_filtered)}")
print(f"删除的样本索引: {indices_to_delete}")

# 读取KEGG通路信息
KEGG_info = pd.read_csv("训练/KEGG_Pathway_information.csv", header=0)

all_gene_list = []
for gene_set in KEGG_info['gene_name']:
    all_gene_list.extend(gene_set.split("/"))

# 处理KEGG基因数据
KEGG_data_dict = dict()
gene_num = 1
for gene in all_gene_list:
    if gene in data_RNA_filtered.columns:  # 使用过滤后的数据
        KEGG_data_dict['gene' + str(gene_num)] = data_RNA_filtered[gene]
        gene_num += 1

# 添加标签列
KEGG_data_dict['label'] = data_RNA_filtered['label']  # 使用过滤后的数据

# 创建新的DataFrame并保存
data_KEGG = pd.DataFrame(KEGG_data_dict)
print(data_KEGG.shape)
data_KEGG.to_pickle(new_path + filename.replace('.csv', '_filtered.pkl'))  # 修改文件名以区分

print("处理完成！已随机删除10%的样本")

xun.csv
原始样本数量: 31
删除样本数量: 3
剩余样本数量: 28
删除的样本索引: ['Pat45' 'Pat41' 'Pat33']
(28, 22955)
处理完成！已随机删除10%的样本


In [193]:
import pandas as pd
import numpy as np
import math
import torch
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from dataset import MyDataSet
from model import Pathway_Guided_Transformer
from utils import train_one_epoch, train_evaluate
from sklearn.metrics import f1_score, recall_score, precision_score
from collections import Counter
 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_integer_mapping(le):
    res = {}
    for cl in le.classes_:
        res.update({cl:le.transform([cl])[0]})
    return res

# 修改数据路径 - 使用删除后的文件
train_data_path = 'data(li)delete10\\xun_filtered.pkl'  # 修改为实际的文件名
KEGG_result = '训练/KEGG_Pathway_information.csv'
model_save_path = 'data(li)delete10/Model_weight/'

# 读取数据
data = pd.read_pickle(train_data_path)
data = data.replace(np.nan, 0)

# 检查删除后的数据
print("删除10%样本后的数据信息:")
print("数据形状:", data.shape)
print("标签分布:")
print(Counter(data['label']))
print("总样本数:", len(data))

x_train = data.iloc[:, :-1].values
x_train = np.log2(x_train+1)
y_train = data.iloc[:,-1]
lbl = LabelEncoder()
y_train = lbl.fit_transform(y_train)
print("标签编码映射:", get_integer_mapping(lbl))

pathway_df = pd.read_csv(KEGG_result, header=0)
pathway_num = list(pathway_df['count'])

batch_size = 64
epochs = 20
lr, lrf = 0.0001, 0.001
max_acc, max_f1, max_recall, max_precision = 0, 0, 0, 0
test_acc, test_f1, test_recall, test_precision  = [], [], [], []

# 根据样本数量调整折数（如果需要）
n_samples = len(data)
if n_samples < 50:
    n_splits = min(3, n_samples)
    print(f"样本数量较少({n_samples})，使用{n_splits}折交叉验证")
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=False)  # 移除random_state
else:
    kfold = StratifiedKFold(n_splits=3, shuffle=False)  # 移除random_sta

KK = 0

for train, val in kfold.split(x_train, y_train):
    print('*'*30, KK, '*'*30)
    loss_m = np.inf
    train_data_mRNA, train_data_y_label = x_train[train,:], y_train[train]
    val_data_mRNA, val_data_y_label = x_train[val,:], y_train[val]

    print(f"训练集大小: {len(train_data_y_label)}, 验证集大小: {len(val_data_y_label)}")

    train_data_mRNA = train_data_mRNA.reshape((train_data_mRNA.shape[0], 1, train_data_mRNA.shape[1]))
    val_data_mRNA = val_data_mRNA.reshape((val_data_mRNA.shape[0], 1, val_data_mRNA.shape[1]))

    train_data_set_KFold = MyDataSet(train_data_mRNA, train_data_y_label)
    val_data_set_KFold = MyDataSet(val_data_mRNA, val_data_y_label)

    train_loader_KFold = torch.utils.data.DataLoader(train_data_set_KFold, 
                                                   batch_size=min(batch_size, len(train_data_set_KFold)), 
                                                   shuffle=True)
    val_loader_KFold = torch.utils.data.DataLoader(val_data_set_KFold, 
                                                 batch_size=min(batch_size, len(val_data_set_KFold)), 
                                                 shuffle=True)

    model = Pathway_Guided_Transformer(
            num_classes = len(np.unique(y_train)),  # 使用实际的类别数
            pathway_number = pathway_num,
            dim = 512,
            depth = 6,
            heads = 8,
            mlp_dim = 1024,
            dropout = 0.1,
            emb_dropout = 0.1,
        ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    lf = lambda x: ((1 + math.cos(x * math.pi / epochs)) / 2) * (1 - lrf) + lrf  # cosine
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)
    criterion = torch.nn.CrossEntropyLoss()
    train_acc_list, val_acc_list, train_loss_list, val_loss_list = [], [], [], []

    for epoch in range(epochs):
        print("-"*50)
        print("epoch:", epoch, "K:", KK)
        train_loss, train_acc = train_one_epoch(now_epoch=epoch, all_epoch=epochs, model=model, optimizer=optimizer, data_loader=train_loader_KFold, 
                                                num=len(train_data_set_KFold), criterion=criterion, device=device)
        scheduler.step()
        val_loss, val_acc, true_label, pre_label = train_evaluate(model=model, data_loader=val_loader_KFold, 
                                                                    num=len(val_data_set_KFold), criterion=criterion, device=device)

        if val_loss<loss_m:
            loss_m = val_loss
            max_acc = val_acc
            max_f1 = f1_score(true_label, pre_label, average='weighted')
            max_recall = recall_score(true_label, pre_label, average='weighted')
            max_precision = precision_score(true_label, pre_label, average='weighted', zero_division=0)
            torch.save(model.state_dict(), model_save_path + "weights_" + str(KK) + ".pth")

        train_acc_list.append(train_acc.item())
        val_acc_list.append(val_acc.item())
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        print("epoch: {}, train loss: {:.8f}, val loss: {:.8f}, train acc: {:.4f}, val acc: {:.4f}".format(epoch, train_loss, val_loss, train_acc, val_acc))
    
    test_acc.append(max_acc.item())
    test_f1.append(max_f1)
    test_recall.append(max_recall)
    test_precision.append(max_precision)
    print("train_acc_list: ", train_acc_list)
    print('val_acc_list: ', val_acc_list)
    print('train_loss_list: ', train_loss_list)
    print('val_loss_list: ', val_loss_list)
    KK = KK+1

print('*'*60)
print("K-fold test_acc: ", test_acc)
print("K-fold test_f1: ", test_f1)
print("K-fold test_recall: ", test_recall)
print("K-fold test_precision: ", test_precision)
print("平均准确率: {:.4f} ± {:.4f}".format(np.mean(test_acc), np.std(test_acc)))
print("平均F1分数: {:.4f} ± {:.4f}".format(np.mean(test_f1), np.std(test_f1)))

删除10%样本后的数据信息:
数据形状: (28, 22955)
标签分布:
Counter({0: 22, 1: 6})
总样本数: 28
标签编码映射: {0: 0, 1: 1}
样本数量较少(28)，使用3折交叉验证
****************************** 0 ******************************
训练集大小: 18, 验证集大小: 10
--------------------------------------------------
epoch: 0 K: 0
train: 18


Epoch 1/20: 100%|██████████████████████| 1/1 [00:02<00:00,  2.53s/it, Loss=1.12]


test: 10
epoch: 0, train loss: 0.06241520, val loss: 0.06501075, train acc: 0.2222, val acc: 0.8000
--------------------------------------------------
epoch: 1 K: 0
train: 18


Epoch 2/20: 100%|███████████████████████| 1/1 [00:02<00:00,  2.48s/it, Loss=0.7]


test: 10
epoch: 1, train loss: 0.03891335, val loss: 0.08633227, train acc: 0.7778, val acc: 0.8000
--------------------------------------------------
epoch: 2 K: 0
train: 18


...

test: 9
epoch: 17, train loss: 0.02602469, val loss: 0.06263081, train acc: 0.7895, val acc: 0.7778
--------------------------------------------------
epoch: 18 K: 2
train: 19


Epoch 19/20: 100%|████████████████████| 1/1 [00:03<00:00,  3.95s/it, Loss=0.507]


test: 9
epoch: 18, train loss: 0.02666600, val loss: 0.06265425, train acc: 0.7895, val acc: 0.7778
--------------------------------------------------
epoch: 19 K: 2
train: 19


Epoch 20/20: 100%|████████████████████| 1/1 [00:04<00:00,  4.08s/it, Loss=0.529]


test: 9
epoch: 19, train loss: 0.02783423, val loss: 0.06265447, train acc: 0.7895, val acc: 0.7778
train_acc_list:  [0.21052631735801697, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7368420958518982, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054, 0.7894737124443054]
val_acc_list:  [0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544, 0.7777777910232544]
train_loss_list:  [0.045813494607021936, 0.03811570217734889, 0.0

In [194]:
torch.save(model.state_dict(), model_save_path + "weights_" + str(KK) + ".model")


In [195]:
print("epoch: {}, train loss: {:.8f}, val loss: {:.8f}, train acc: {:.4f}, val acc: {:.4f}".format(epoch, train_loss, val_loss, train_acc, val_acc))

epoch: 19, train loss: 0.02783423, val loss: 0.06265447, train acc: 0.7895, val acc: 0.7778


In [196]:
import pandas as pd

base_path = 'data(li)delete10/训练/'
new_path = 'data(li)delete10/训练/'
# 将 filelist 替换为 exp.csv
filename = 'te_xun.csv'

KEGG_info = pd.read_csv("训练/KEGG_Pathway_information.csv", header=0)

all_gene_list = []
for gene_set in KEGG_info['gene_name']:
    all_gene_list.extend(gene_set.split("/"))

# 读取 exp.csv 文件
print(filename)
data_RNA = pd.read_csv(base_path + filename,index_col=0)  # 使用 read_csv 读取 CSV 文件
KEGG_data_dict = dict()
gene_num = 1
for gene in all_gene_list:
    if gene in data_RNA.columns:  # 检查基因是否在数据框中
        KEGG_data_dict['gene' + str(gene_num)] = data_RNA[gene]
        gene_num += 1

KEGG_data_dict['label'] = data_RNA['label']  # 假设 'label' 列在数据框中
data_KEGG = pd.DataFrame(KEGG_data_dict)
print(data_KEGG)
data_KEGG.to_pickle(new_path + filename.replace('.csv', '.pkl'))  # 保存为 pkl 文件

te_xun.csv
           gene1      gene2     gene3      gene4      gene5      gene6  \
Pat03   0.423672  11.349284  0.479430   9.926252  23.168080   1.277639   
Pat28   0.677110  34.613793  0.294675  12.827103  35.033630   4.919835   
Pat49   1.084989  15.863534  0.258371  16.475945  40.957265   1.602234   
Pat38   0.272135  17.157354  0.184135  34.860374  41.482285  12.017714   
Pat02   0.297357   6.906578  0.152635  17.407315  37.292708   6.656415   
Pat118  2.163818  14.388787  0.631081   7.674412  31.249620   7.233839   
Pat36   0.327394  29.165064  0.520166  11.338593  34.583755   0.517158   
Pat43   0.966011  15.781033  0.299822  18.233375  36.627173   3.285748   

            gene7      gene8      gene9    gene10  ...  gene22946  gene22947  \
Pat03   86.309893  15.897700  64.521477  0.872070  ...  11.670901   2.591405   
Pat28   18.941887   0.067197  14.813769  2.363295  ...  13.372190   0.957363   
Pat49   36.210708   2.553129  27.195108  2.331151  ...  13.135285   3.612381   
Pa

In [197]:
import pandas as pd
import torch
from model import Pathway_Guided_Transformer
from utils import test_TCGA, test_ICGC, test_GEO_p_m, test_GEO_m, test_GEO_p

# Test data path
test_data_path = 'data(li)delete10/训练/'
path_test_data_TCGA = test_data_path + 'te_xun.pkl'
# Model weight file
path_weight = 'data(li)delete10\\Model_weight\\weights_2.pth'

# prediction result
prediction_path = 'data(li)delete10/训练/'
TCGA_prediction = prediction_path + 'prediction.csv'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 64
pathway_df = pd.read_csv('RNAseq/KEGG_Pathway_information.csv', header=0)
pathway_num = list(pathway_df['count'])
model = Pathway_Guided_Transformer(
        num_classes = 2,
        pathway_number = pathway_num,
        dim = 512,
        depth = 6,
        heads = 8,
        mlp_dim = 1024,
        dropout = 0.1,
        emb_dropout = 0.1,
    ).to(device)
print("*"*20, "test for TCGA", "*"*20)
TCGA_acc = test_TCGA(path_test_data_TCGA,  path_weight, TCGA_prediction, model, batch_size, device)

******************** test for TCGA ********************


d:\比较方法\比较方法\BPformer（新数据）\utils.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path_weight))


test: 8
x_mRNA shape: torch.Size([8, 1, 22954])
y_label: ('0', '0', '0', '1', '0', '0', '0', '0')


In [198]:


# Test data path
test_data_path = 'data(li)delete10/外部测试/'
path_test_data_TCGA = test_data_path + 'test.pkl'


In [199]:
# Model weight file
path_weight = 'data(li)delete10\\Model_weight\\weights_2.pth'

# prediction result
prediction_path = 'data(li)delete10/外部测试/'
TCGA_prediction = prediction_path + 'prediction.csv'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 64
pathway_df = pd.read_csv('RNAseq/KEGG_Pathway_information.csv', header=0)
pathway_num = list(pathway_df['count'])


In [200]:
print("*"*20, "test for TCGA", "*"*20)
TCGA_acc = test_TCGA(path_test_data_TCGA,  path_weight, TCGA_prediction, model, batch_size, device)

******************** test for TCGA ********************


d:\比较方法\比较方法\BPformer（新数据）\utils.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path_weight))


test: 16
x_mRNA shape: torch.Size([16, 1, 22954])
y_label: ('0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0')
